In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [4]:
from torch import Tensor
import torch

from typing import Tuple

In [78]:
# Operation with multiple inputs, gradients
class Operation(object):

    def __init__(self):
        pass

    def forward(self, *inputs) -> Tuple[Tensor]:

        self.inputs = inputs

        self.outputs = self._outputs()

        return self.outputs


    def backward(self, *output_grads) -> Tuple[Tensor]:

        assert_same_shapes(self.outputs, output_grads)

        self._compute_grads(output_grads)

        assert_same_shapes(self.inputs, self.input_grads)
        return self.input_grads


    def _compute_grads(self, output_grads: Tuple[Tensor]) -> Tensor:
        
        self.input_grads = self._input_grads(output_grads)

        return self.input_grads

    def _outputs(self) -> Tuple[Tensor]:
        raise NotImplementedError()

    def _input_grads(self, *output_grads) -> Tuple[Tensor]:
        raise NotImplementedError()

In [79]:
# ParamOperation with multiple gradients
class ParamOperation(Operation):

    def __init__(self, param: Tensor) -> Tensor:
        super().__init__()
        self.param = param


    def _compute_grads(self, output_grads: Tuple[Tensor]) -> Tensor:
        self.input_grad = self._input_grad(output_grads)
        self.param_grad = self._param_grad(output_grads)


    def _param_grad(self, output_grad: Tensor) -> Tensor:
        raise NotImplementedError()



In [148]:
# Multiply class
class Multiply(Operation):
   
    def __init__(self):
        pass

    def _outputs(self) -> Tuple[Tensor]:
        '''
        Element-wise multiplication
        '''
        assert len(self.inputs) == 2
        
        return self.inputs[0] * self.inputs[1]

    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:

        return self.inputs[1] * output_grads[0],\
               self.inputs[0] * output_grads[0]
     

In [153]:
# assert_same_shapes function
def assert_same_shapes(tensors: Tuple[Tensor],
                       tensor_grads: Tuple[Tensor]):
    if len(tensors) == 1:
        tensors = tensors[0]
    if len(tensor_grads) == 1:
        tensor_grads = tensor_grads[0]        

    for tensor, tensor_grad in zip(tensors, tensor_grads):
        assert tensor.shape == tensor_grad.shape, \
        '''
        Two tensors should have the same shape; instead, first Tensor's shape is {0}
        and second Tensor's shape is {1}.
        '''.format(tuple(tensor_grad.shape), tuple(tensor.shape))
        return None

In [154]:
# define inputs to Multiply Operation
m = Multiply()
torch.manual_seed(102218)
m1, m2 = torch.rand(2,2), torch.rand(2,2)

In [155]:
m.forward(m1, m2) # tuple of length 1

tensor([[0.8841, 0.2608],
        [0.1206, 0.4546]])

In [156]:
m_grad = torch.ones_like(m1)
m.backward(m_grad)

(tensor([[0.9344, 0.4609],
         [0.4948, 0.5119]]), tensor([[0.9462, 0.5658],
         [0.2438, 0.8881]]))

## PyTorch `Multiply`

### Without the class

In [157]:
print(m1)
print(m1.requires_grad)

tensor([[0.9462, 0.5658],
        [0.2438, 0.8881]])
False


In [158]:
# detach to copy values
m1_g = m1.detach()
m2_g = m2.detach()

In [159]:
# require gradients
m1_g.requires_grad = True
m2_g.requires_grad = True

In [160]:
# perform operation
out = m1_g * m2_g

In [161]:
# define grad
mul_grad = torch.ones_like(out)

In [162]:
# send gradient backward
out.backward(gradient=mul_grad)

In [163]:
# same gradients as above!
print(m1_g.grad)
print(m2_g.grad)

tensor([[0.9344, 0.4609],
        [0.4948, 0.5119]])
tensor([[0.9462, 0.5658],
        [0.2438, 0.8881]])


### `PyTorchOperation` base class

In [164]:
# PyTorch Operation class
class PyTorchOperation(Operation):

    def __init__(self) -> Tensor:
        super().__init__()


    def forward(self, *inputs) -> Tuple[Tensor]:

        self.inputs = inputs

        self.inputs_with_grad = self._inputs_autograd()
        
        self.outputs = self._outputs()

        return self.outputs
        
        
    def _outputs(self) -> Tuple[Tensor]:
        raise NotImplementedError
        

    def _inputs_autograd(self) -> Tuple[Tensor]:
        inputs_with_grad = tuple(inp.detach() for inp in self.inputs)
        for inp in inputs_with_grad:
            inp.requires_grad = True
        return inputs_with_grad
        

    def _input_grads(self, output_grads: Tensor) -> Tensor:

        for out, grad in zip(self.outputs, output_grads):
            out.backward(gradient=grad)

        input_grads = tuple()
        for inp in self.inputs_with_grad:
            input_grads = input_grads + (inp.grad,)
        
        return input_grads

In [165]:
# Multiply_PyTorch class
class Multiply_PyTorch(PyTorchOperation):
   
    def __init__(self):
        super().__init__()

    def _outputs(self) -> Tuple[Tensor]:
        
        assert len(self.inputs) == 2
        
        return self.inputs_with_grad[0] * self.inputs_with_grad[1],

In [166]:
# Initialize Multiply stuff
m = Multiply_PyTorch()
mp1, mp2 = torch.rand(2,2), torch.rand(2,2)

In [167]:
print(mp1, mp2)

tensor([[0.3988, 0.1799],
        [0.6886, 0.0705]]) tensor([[0.3258, 0.4976],
        [0.6935, 0.2254]])


In [168]:
# Forward
m.forward(mp1, mp2)

(tensor([[0.1299, 0.0895],
         [0.4775, 0.0159]], grad_fn=<ThMulBackward>),)

In [169]:
# Initialize grad and backward
m.backward(torch.ones_like(mp1))

(tensor([[0.3258, 0.4976],
         [0.6935, 0.2254]]), tensor([[0.3988, 0.1799],
         [0.6886, 0.0705]]))

## More complicated example

* Divide two inputs into:
    * `A`, `B`
    * `A` gets copied into `A1` and `A2`
    * `A1` and `A2` each get multiplied, bias added, activation, to create `C1` and `C2`.
    * These get added together to create `D`.
    * `B` and `D` get multiplied to create `E`. 
    
Then the question is: what is the gradient of `E` with respect to `A`.

### Without class

In [207]:
# Copy class
class Copy(Operation):
   
    def __init__(self, num=2):
        self.num = num

    def _outputs(self) -> Tuple[Tensor]:
        '''
        Element-wise multiplication
        '''
        output = tuple()
        for i in range(self.num):
            output = output + self.inputs
            
        return output

    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:
        input_grad = torch.zeros_like(output_grads[0])
        for grad in output_grads:
             input_grad = input_grad + grad
        return input_grad, 

In [208]:
class Add2(Operation):
   
    def __init__(self):
        pass

    def _outputs(self) -> Tuple[Tensor]:
        '''
        Element-wise addition
        '''
        assert len(self.inputs) == 2
        
        return self.inputs[0] + self.inputs[1]

    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:

        return output_grads[0], output_grads[0]
     

In [209]:
# initialize tensors
torch.manual_seed(102218)
a, b = torch.rand(2,2), torch.rand(2,2)
w1, w2 = torch.rand(2,2), torch.rand(2,2)
b1, b2 = torch.rand(1,2), torch.rand(1,2)

In [210]:
# set requires_grad = True for all
for t in [a, b, w1, w2, b1, b2]:
    t.requires_grad = True

In [211]:
# define operations
c = Copy()
add2 = Add2()
mul = Multiply()
wm1 = WeightMultiply(w1)
ba1 = BiasAdd(b1)
wm2 = WeightMultiply(w2)
ba2 = BiasAdd(b2)

In [212]:
# define forward path
a1, a2 = c.forward(a)
c1 = ba1.forward(wm1.forward(a1))
c2 = ba2.forward(wm2.forward(a2))
d = add2.forward(c1, c2)
e = mul.forward(b, d)

In [213]:
e # output

tensor([[2.4230, 1.1577],
        [1.2516, 1.0909]], grad_fn=<ThMulBackward>)

In [214]:
torch.sum(e) # sum

tensor(5.9232, grad_fn=<SumBackward0>)

In [215]:
a.grad # None

In [216]:
# passing gradient backwards
e.backward(gradient=torch.ones_like(e))

In [217]:
# all grads work
print(a.grad)

print(b.grad)

print(w1.grad)

print(b1.grad)

tensor([[0.9893, 1.4278],
        [0.7053, 0.8353]])
tensor([[2.5932, 2.5116],
        [2.5298, 2.1312]])
tensor([[1.0048, 0.5609],
        [0.9681, 0.7154]])
tensor([[1.4291, 0.9728]])


This implies that increasing `a[0][0]` by `0.1` will increase the sum from `5.9232` to `5.9232 + 0.1 * 0.9893`.

In [228]:
# define what sum with "a_new" should be: 
5.9232 + 0.1 * 0.9893

6.02213

In [229]:
# define helper function
def sum_with_a(a):
    a1, a2 = c.forward(a)
    c1 = ba1.forward(wm1.forward(a1))
    c2 = ba2.forward(wm2.forward(a2))
    d = add2.forward(c1, c2)
    e = mul.forward(b, d)
    return torch.sum(e)

In [225]:
# define new version of a with one value incremented
a_new = a.clone()
a_new[0][0] += 0.1
print(a_new)

In [230]:
print(sum_with_a(a))
print(sum_with_a(a_new))

tensor(5.9232, grad_fn=<SumBackward0>)
tensor(6.0222, grad_fn=<SumBackward0>)


Works!

### Turning this into a class

`Layer` class:

* Has forward and backward methods. These simply loop through the operations, passing things forwards or backards.
* On setup, requires a `num_in`

`AutogradBlock` class:

* Has forward method, which calls the `_output` method and `_setup` if it is the first iteration.
* On setup, requires we define all ops in a `Dict[Operation]`. In addition, we define weights and give them a gradient.
* On `_output`, we actually compute the output from the inputs.
* We get the `param_grads` in a similar way we get them from `Layer`s - looping through a dictionary now instead of through a list.

Trouble is: this solution begs the question: why not write an autograd library and do everything that way?

Options for autograd from scratch:

* Could invest a week and unpack the example at `autodiff` in README.
   * He makes heavy use of `einsum`, which is its own overhead.
* Just use PyTorch (bad - why the operations? Might be ok to illustrate LSTM). 

Point is to illustrate how things work. Period.

In [324]:
from lincoln.operations.operations import WeightMultiply
from lincoln.operations.operations import BiasAdd

In [424]:
# AutogradBlock with multiple gradients
class AutogradBlock(object):

    def __init__(self) -> Tensor:
        super().__init__()
        self.params: Dict[Tensor] = {}
        self.ops: Dict[Operation] = {}
        self.first: bool = True

            
    def _setup_block(self) -> Tuple[Tensor]:
        pass
        

    def forward(self, *inputs) -> Tuple[Tensor]:

        if self.first:
            self._setup_block()
            
        self.inputs = inputs
        
        self.inputs_with_grad = self._inputs_autograd()

        self.outputs = self._outputs()

        return self.outputs

    def _inputs_autograd(self) -> Tuple[Tensor]:
        inputs_with_grad = tuple(inp.detach() for inp in self.inputs)
        for inp in inputs_with_grad:
            inp.requires_grad = True
        return inputs_with_grad
    
    
    def backward(self, *output_grads) -> Tuple[Tensor]:

        assert_same_shapes(self.outputs, output_grads)

        self.input_grads = self._input_grads(output_grads)
    
        assert_same_shapes(self.inputs, self.input_grads)
        return self.input_grads


    def _outputs(self) -> Tuple[Tensor]:
        raise NotImplementedError()


    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:
        
        if len(output_grads) == 1:
            self.outputs.backward(output_grads)
        else: 
            for out, grad in zip(self.outputs, output_grads):
                out.backward(gradient=grad)

        input_grads = tuple()
        for inp in self.inputs_with_grad:
            input_grads = input_grads + (inp.grad,)
        
        return input_grads



In [468]:
# ToyExample class
class ToyExample(AutogradBlock):
   
    def __init__(self, 
                 seed=12345):
        super().__init__()
        self.seed = seed
    
        
    def _setup_block(self) -> Tuple[Tensor]:

        torch.manual_seed(self.seed)
        self.params['w1'] = torch.rand(2,2)
        self.params['w2'] = torch.rand(2,2)
        self.params['b1'] = torch.rand(1,2)
        self.params['b2'] = torch.rand(1,2)
        
        self.ops['cp'] = Copy()
        self.ops['add2'] = Add2()
        self.ops['mul'] = Multiply()
        self.ops['wm1'] = WeightMultiply(self.params['w1'])
        self.ops['ba1'] = BiasAdd(self.params['b1'])        
        self.ops['wm2'] = WeightMultiply(self.params['w2'])
        self.ops['ba2'] = BiasAdd(self.params['b2'])
        
    def _outputs(self) -> Tuple[Tensor]:
        
        a, b = self.inputs_with_grad 
        a1, a2 = self.ops['cp'].forward(a) 
        c1 = self.ops['ba1'].forward(self.ops['wm1'].forward(a1))
        c2 = self.ops['ba2'].forward(self.ops['wm2'].forward(a2))
        d = self.ops['add2'].forward(c1, c2)
        
        return self.ops['mul'].forward(b, d)

In [469]:
# instantiate
t = ToyExample(seed=102218)

In [470]:
# check a and b
print(a)
print(b)

tensor([[0.9462, 0.5658],
        [0.2438, 0.8881]])
tensor([[0.9344, 0.4609],
        [0.4948, 0.5119]])


In [471]:
# new input tensors
torch.manual_seed(102218)
a = torch.rand(2,2)
b = torch.rand(2,2)

In [472]:
# result
out = t.forward(a, b)
print(out)
print(torch.sum(out))

tensor([[3.0692, 0.9283],
        [1.0894, 0.8928]], grad_fn=<ThMulBackward>)
tensor(5.9798, grad_fn=<SumBackward0>)


In [473]:
a.grad # no grad

In [474]:
t.backward(torch.ones_like(out))

(tensor([[2.2304, 1.3354],
         [1.4560, 1.0821]]), tensor([[3.2847, 2.0140],
         [2.2020, 1.7442]]))

In [475]:
a.grad # still no grad

#### Testing gradients

In [479]:
# new version of a
a_new = a.clone()
a_new[0][0] += 0.1
print(a_new)

tensor([[1.0462, 0.5658],
        [0.2438, 0.8881]])


In [480]:
out_new = t.forward(a_new, b)
print(out_new)
print(torch.sum(out_new))

tensor([[3.2449, 0.9756],
        [1.0894, 0.8928]], grad_fn=<ThMulBackward>)
tensor(6.2028, grad_fn=<SumBackward0>)


In [481]:
# predicted value
# old_sum + 0.1 * gradient
5.9798 + 0.1 * 2.2304

6.20284

Works!

### LSTMs and beyond

LSTM Node will be an `AutogradBlock` with three inputs and three outputs. 



# LSTM Operations

## `LSTM Layer` 

Input: series of word embeddings, cell state.

Output: series of embeddings of the same size as the word, hidden state.

Cell state will be an attribute of the layer.

LSTM layer will have an additional function "reset state" that resets the hidden state to zero.

## `LSTM Node` 

Each LSTM layer will have a series of a special kind of operation, `LSTM Node`. These nodes will have a series of special operations: they will take in as input an embedding and a hidden state and pass out an embedding and an updated hidden state. 

Operations of LSTM Node:

Receives as input:

* X (batch size x embedding dim)
* H_prev (batch size x hidden dim)
* C_prev (batch size x hidden dim)

Diff components:

Create "Z"
Copy "Z"


1. Z = Concat(X, H)
1. Z1, Z2, Z3, Z4 = Copy(Z)
1. F = WeightMultiply(Z1, W_f)
1. F = BiasAdd(F, B_f)
1. F_out = Sigmoid(F)
1. I = WeightMultiply(Z2, W_i)
1. I = BiasAdd(I, B_i)
1. I_out = Sigmoid(I)
1. C = WeightMultiply(Z3, W_c)
1. C = BiasAdd(C, B_c)
1. C_bar = Tanh(C, B_c)
1. C1 = Multiply(F_out, C_prev)
1. C2 = Multiply(I_out, C_bar)
1. C_new = Add(C1, C2)
1. O = WeightMutiply(Z4, W_o)
1. O = Add(O, B_o)
1. O_out = Sigmoid(O)
1. C_tan = Tanh(C_new)
1. H_new = Multiply(O_out, C_tan)
1. H_out = WeightMultiply(H_new, W_v)
1. X_out = BiasAdd(H_out, B_v)

Issues:

* Concat operations needs to produce four distinct outputs. 
    * Potential solution: `Copy` operation that sums gradients.

* Need a way to handle branching:
* After `Copy`, there will be four `forward` operations happening.
* Can't write `for operation in self.operations: operation.forward(X)`
    * Maybe I can

That's it!        
1. Parameters are initialized at the `LSTMLayer` level. 
2. `LSTMNode` has a series of `ParamOperation`s.
3. At the beginning of each operation's `forward` and `backward` methods, the nodes get parameters from the `LSTMLayer`.
4. On the backward pass, they:
    * Provide `param_grad`s for each `ParamOperation`.
    * `LSTMNode` will need a function to return all of its `param_grad`s.
5. The `LSTMLayer` can get the `param_grad`s from all of the `LSTMNode`s, aggregate them.


In [ ]:
class LSTMNode(object):

    def __init__(self):

    
    
    def forward(self, 
                X_input: Tensor, 
                H_in: Tensor,
                C_in: Tensor, 
                params: Dict[Tensor]):
        
        operations_1 = [
            Concat(X_input, H_in),
            Copy(4), 
        ] 
        
        operations_2 = [
            WeightMultiply(params['W_f']),
            BiasAdd(params['B_f']),
            Sigmoid()
        ] # outputs F_out
        
        operations_3 = [
            WeightMultiply(params['W_i']),
            BiasAdd(params['B_i']),
            Sigmoid()
        ] # outputs I_out
        
        operations_4 = [
            WeightMultiply(params['W_c']),
            BiasAdd(params['B_c']),
            Tanh()
        ] # outputs C_bar
        
        operations_5 = [
            Multiply(), # outputs C1
            Multiply()
        ]

        operation_blocks = [
            [operations_1, 
             [operations_2, operations_3]
             , operations_4, oper]
            
        ]
        self.input = input

        self.output = self._output()

        return self.output


    def backward(self, output_grad: Tensor) -> Tensor:

        assert_same_shape(self.output, output_grad)

        self._compute_grads(output_grad)

        assert_same_shape(self.input, self.input_grad)
        return self.input_grad


    def _compute_grads(self, output_grad: Tensor) -> Tensor:

        self.input_grad = self._input_grad(output_grad)

        assert_same_shape(self.input, self.input_grad)
        return self.input_grad

    def _output(self) -> Tensor:
        raise NotImplementedError()

    def _input_grad(self, output_grad: Tensor) -> Tensor:
        raise NotImplementedError()

```python
Z, Z = Concat(X, H_in) # Concat takes in two inputs produces one output, will have X_grad and H_in_grad

Z_1, Z_2, Z_3 = Copy(Z, 3)

F = WeightMultiply(Z_1, W_f) 
F = BiasAdd(F, B_f)
F_out = Sigmoid(F)

I = WeightMultiply(Z_2, W_i) 
I = BiasAdd(I, B_i)
I_out = Sigmoid(I)

C = WeightMultiply(Z_3, W_c)
C = BiasAdd(C, B_c)
C_bar = Tanh(C)



Operations are the smallest units that have inputs and outputs defined by having a input and output.

Going to need to build operations that:

* Can take in one input and produce multiple outputs
* If it produces multiple outputs, _gradients will need to accumulate_.

## Toy example

* Divide two inputs into:
    * `A`, `B`
    * `A` gets copied into `A1` and `A2`
    * `A1` and `A2` each get multiplied, bias added, activation, to create `C1` and `C2`.
    * These get added together to create `D`.
    * `B` and `D` get multiplied to create `E`. 
    
Then the question is: what is the gradient of `E` with respect to `A`.

In [8]:
def mult(*args):
    num = 1
    print(type(args))
    for arg in args:
        num *= arg
    return num

In [9]:
mult(3)

<class 'tuple'>


3

In [10]:
from typing import Tuple
from torch import Tensor

In [11]:
def assert_same_shapes(tensors: Tuple[Tensor],
                       tensor_grads: Tuple[Tensor]):
    for tensor, tensor_grad in zip(tensors, tensor_grads):
        assert tensor.shape == tensor_grad.shape, \
        '''
        Two tensors should have the same shape; instead, first Tensor's shape is {0}
        and second Tensor's shape is {1}.
        '''.format(tuple(tensor_grad.shape), tuple(tensor.shape))
        return None

#### New `Operation` class

In [61]:
class Operation(object):

    def __init__(self):
        pass

    def forward(self, *inputs) -> Tuple[Tensor]:

        self.inputs = inputs

        self.outputs = self._outputs()

        return self.outputs


    def backward(self, *output_grads) -> Tuple[Tensor]:
#         import pdb; pdb.set_trace()
        assert_same_shapes(self.outputs, output_grads)

        self._compute_grads(output_grads)

        assert_same_shapes(self.inputs, self.input_grads)
        return self.input_grads


    def _compute_grads(self, output_grads: Tuple[Tensor]) -> Tensor:
        
        self.input_grads = self._input_grads(output_grads)

        return self.input_grads

    def _outputs(self) -> Tuple[Tensor]:
        raise NotImplementedError()

    def _input_grads(self, *output_grads) -> Tuple[Tensor]:
        raise NotImplementedError()


In [18]:
a = Tensor([1,2])
b = Tensor([3,4])
a * b

tensor([3., 8.])

#### `Multiply`

In [19]:
class Multiply(Operation):
   
    def __init__(self):
        pass

    def _outputs(self) -> Tuple[Tensor]:
        '''
        Element-wise multiplication
        '''
        assert len(self.inputs) == 2
        
        return (self.inputs[0] * self.inputs[1],)

    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:

        return self.inputs[1] * output_grads[0],\
               self.inputs[0] * output_grads[0]
     

In [20]:
m = Multiply()

In [21]:
print(m.forward(a,b))
print(m.backward(Tensor([1,1])))

(tensor([3., 8.]),)
(tensor([3., 4.]), tensor([1., 2.]))


#### `Add2`

In [33]:
class Add2(Operation):
   
    def __init__(self):
        pass

    def _outputs(self) -> Tuple[Tensor]:
        '''
        Element-wise addition
        '''
        assert len(self.inputs) == 2
        
        return (self.inputs[0] + self.inputs[1],)

    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:

        return output_grads[0], output_grads[0]
     

In [34]:
add = Add2()

In [36]:
print(add.forward(a,b))
print(add.backward(torch.ones_like(a)))

(tensor([[ 6.,  8.],
        [10., 12.]]),)
(tensor([[1., 1.],
        [1., 1.]]), tensor([[1., 1.],
        [1., 1.]]))


#### `Concat`

In [22]:
a = Tensor([[1,2],[3,4]])
b = Tensor([[5,6],[7,8]])

In [23]:
import torch

In [24]:
c = torch.cat([a,b], dim=1)

In [25]:
torch.split(c, [2, 2], dim=1)

(tensor([[1., 2.],
         [3., 4.]]), tensor([[5., 6.],
         [7., 8.]]))

In [26]:
tuple(inp.shape[1] for inp in (a,b))

(2, 2)

In [27]:
class Concat2(Operation):
   
    def __init__(self):
        pass

    def _outputs(self) -> Tuple[Tensor]:
        '''
        Element-wise multiplication
        '''
        assert len(self.inputs) == 2
        
        self.input_shapes = [inp.shape[1] for inp in self.inputs]
        
        return (torch.cat([a,b], dim=1), )

    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tuple[Tensor]:
        return torch.split(output_grads[0], 
                           self.input_shapes,
                           dim=1)
     

In [45]:
c = Concat2()
out = c.forward(a,b)
print(out)
c.backward(torch.ones_like(out[0]))

(tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]]),)


(tensor([[1., 1.],
         [1., 1.]]), tensor([[1., 1.],
         [1., 1.]]))

In [48]:
torch.add?

In [65]:
cp = Copy()
out = cp.forward(a)
cp.backward(a, a)

(tensor([[2., 4.],
         [6., 8.]]),)

* Divide two inputs into:
    * `A`, `B`
    * `A` gets copied into `A1` and `A2`
    * `A1` and `A2` each get multiplied, bias added, activation, to create `C1` and `C2`.
    * These get added together to create `D`.
    * `B` and `D` get multiplied to create `E`. 
    
Then the question is: what is the gradient of `E` with respect to `A`.

In [115]:
inputs = a, b

In [226]:
from collections import OrderedDict

In [227]:
OrderedDict?

In [ ]:
from itertools import OrderedDict

In [254]:
ops_1 = [Copy()]
ops_2 = [[[WeightMultiply(w1), BiasAdd(b1), Sigmoid()],
          [WeightMultiply(w1), BiasAdd(b1), Sigmoid()]],
         Add2()]
ops_3 = [Multiply()]

In [255]:
forward_op(ops_1, a)

(tensor([[1., 2.],
        [3., 4.]]),)


((tensor([[1., 2.],
          [3., 4.]]),), (tensor([[1., 2.],
          [3., 4.]]),))

Three cases:

* If `op` is an operation, simply call forward.
* If `op` is a list and `op[0]` is an operation, call `forward_op` on all the operations in `op`
* If `op` is a list and `op[0]` is a list, call `forward_op` on `op` and `inputs`.

In [274]:
def forward_op(ops, *inputs):
    out = inputs
    for op in ops:
        if isinstance(op, list):
            if isinstance(op[0], list):
                out = tuple(forward_op(operation, inp) for operation, inp in zip(op, out))
            else:
                

            
        if len(out) == 1:
            out = op.forward(out[0])
        else:
            out = op.forward(out)
        
    return out

If type of op is list, that means we want to apply the first element of the list to the 

In [275]:
forward_op(ops_2, forward_op(ops_1, a))

(tensor([[1., 2.],
        [3., 4.]]),)
[[<__main__.WeightMultiply object at 0x1208a4f28>, <__main__.BiasAdd object at 0x1208a4b70>, Sigmoid], [<__main__.WeightMultiply object at 0x1208a4dd8>, <__main__.BiasAdd object at 0x1208a4da0>, Sigmoid]]
((tensor([[1., 2.],
        [3., 4.]]), tensor([[1., 2.],
        [3., 4.]])),)
2
2
[<__main__.WeightMultiply object at 0x1208a4f28>, <__main__.BiasAdd object at 0x1208a4b70>, Sigmoid]
((tensor([[1., 2.],
        [3., 4.]]), tensor([[1., 2.],
        [3., 4.]])),)
3
2


AssertionError: 

In [234]:

e = forward_op(ops_3, b, forward_op(ops_2, forward_op(ops_1, a)))

AttributeError: 'list' object has no attribute 'forward'

In [221]:
def mult(start, *args):
    res = start
    for num in args:
        res *= num
    return res

Good. Now how to do the backward pass?

In [89]:
class ParamOperation(Operation):

    def __init__(self, param: Tensor) -> Tensor:
        super().__init__()
        self.param = param


    def _compute_grads(self, output_grads: Tuple[Tensor]) -> Tensor:
        self.input_grad = self._input_grad(output_grads)
        self.param_grad = self._param_grad(output_grads)


    def _param_grad(self, output_grad: Tensor) -> Tensor:
        raise NotImplementedError()


In [125]:
class WeightMultiply(ParamOperation):

    def __init__(self, W: Tensor):
        super().__init__(W)

    def _outputs(self) -> Tensor:
        return (torch.mm(self.inputs[0], self.param), )

    def _input_grads(self, output_grad: Tensor) -> Tensor:
        return (torch.mm(output_grad[0], self.param.transpose(0, 1)), )

    def _param_grad(self, output_grad: Tensor) -> Tensor:
        return torch.mm(self.inputs[0].transpose(0, 1), output_grad[0])


class BiasAdd(ParamOperation):

    def __init__(self,
                 B: Tensor):
        super().__init__(B)


    def _outputs(self) -> Tensor:
        return (torch.add(self.inputs[0], self.param), )


    def _input_grads(self, output_grads: Tuple[Tensor]) -> Tensor:
        return (torch.ones_like(self.inputs[0]) * output_grad[0], )


    def _param_grad(self, output_grads: Tuple[Tensor]) -> Tensor:
        param_grad = torch.ones_like(self.param) * output_grad[0]
        return torch.sum(param_grad, dim=0).reshape(1, param_grad.shape[1])

In [126]:
class Sigmoid(Operation):
    '''
    Sigmoid activation function
    '''
    def __init__(self) -> None:
        super().__init__()


    def _outputs(self) -> Tuple[Tensor]:
        return (1.0/(1.0+torch.exp(-1.0 * self.input)), )


    def _input_grad(self, output_grad: Tensor) -> Tensor:
        sigmoid_backward = self.output * (1.0 - self.output)
        input_grad = sigmoid_backward * output_grad
        return (input_grad, )

    def __repr__(self):
        return "Sigmoid"

First: copy, and then weight multiply each.

In [91]:
torch.rand?

In [127]:
torch.manual_seed(101218)
op1 = Copy()
w1, w2 = torch.rand(2,2), torch.rand(2,2)
b1, b2 = torch.rand(1,2), torch.rand(1,2)
op2, op3 = WeightMultiply(w1), WeightMultiply(w2)
ops2 = [op2, op3]
stages = [[op1], ops2]

In [120]:
stage_in = (a,)
for stage in stages:
    stage_out = tuple()
    for i, op in enumerate(stage):
        stage_out = stage_out + op.forward(stage_in[i])
    stage_in = stage_out
final_out = stage_out

In [121]:
def ones_like_multiple(output: Tuple[Tensor]) -> Tuple[Tensor]:
    return tuple(torch.ones_like(tensor) for tensor in output)

In [153]:
full_ops

[[<__main__.Copy at 0x12063a390>,
  [[<__main__.WeightMultiply at 0x12063a3c8>,
    Sigmoid],
    Sigmoid]],

In [196]:
full_ops = [[[WeightMultiply(w1), BiasAdd(b1), Sigmoid()], 
             [WeightMultiply(w1), BiasAdd(b1), Sigmoid()]], Add2()]

Step 1: full_ops[0]

In [197]:
len(full_ops)

2

In [198]:
full_ops[0]

[[<__main__.WeightMultiply at 0x12072ca58>,
  Sigmoid],
  Sigmoid]]

We see a list of lists. It has length 2. 

That means ultimately, we want the first input to go through the first argument, and the second input to go through the second.

In [199]:
full_ops[0][0]

 Sigmoid]

Now we have a non-nested list. This signals: take the first argument, and feed it through.

In [200]:
full_ops[0][1]

 Sigmoid]

Here we have another non-nested list. We must similarly: take the second argument, and feed it through. 

Now we'll have two outputs, at the end of `full_ops[0]`. We move on the `full_ops[1]`, which indeed is:

In [201]:
full_ops[1]

In [203]:
def is_nested_list(nested_list):
    for el in nested_list:
        if isinstance(el, list):
            return True
    return False

#### Manual walkthrough: `full_ops_advanced`

In [218]:
full_ops_advanced = [Copy(), 
  [[WeightMultiply(w1), BiasAdd(b1), Sigmoid()], 
   [WeightMultiply(w1), BiasAdd(b1), Sigmoid()]],
  Add2()],
 Multiply()]

In [207]:
full_ops_advanced

[[<__main__.Copy at 0x12063a320>,
  [[<__main__.WeightMultiply at 0x12063a160>,
    Sigmoid],
    Sigmoid]],

We have a list of length 2. This means: pass the first element to the first branch, and the second element to the second branch.

In [208]:
full_ops_advanced[0]

 [[<__main__.WeightMultiply at 0x12063a160>,
   Sigmoid],
   Sigmoid]],

This is still a nested list. However, the first element is an operation. So, we want to start by feeding this element through the `Operation`.

In [210]:
full_ops_advanced[0][0]

This outputs two things, and indeed:

In [211]:
full_ops_advanced[0][1]

[[<__main__.WeightMultiply at 0x12063a160>,
  Sigmoid],
  Sigmoid]]

Is of length two.

So, we have:

* If the first element in the list is an operation, feed the input through it, and keep doing so until you reach an element in the list that is not an `Operation`.
* At some point, you'll have outputs from the `Operation`s, and another (possibly nested) list of operations to deal with. At this point, you should call the main function again, with the `output`s from the first part, and the new operations. This function should return a number of outputs equal to the length of the list.
* At this point, we'll have two outputs, and we'll move on to:

In [219]:
full_ops_advanced[0][2]

We'll take the outputs from the prior section and feed them through this function.

Finally, we reach another function. Since this function is next in the list, our code will have to assume that it 

This nested list structure cannot support branching. Need to try another approach.

In [216]:
full_ops_advanced[1]

# New approach: using PyTorch's `autograd` feature